In [1]:
import pandas as pd
import os
import numpy as np
import cv2
from utils import draw_tracking_result, save_tracking_results, load_detections
from tracks import track_iou, create_similarity_matrix, associate_detections_to_tracks, update_tracks, initialize_new_track, look_forward, init_model
from KalmanFilter import KalmanFilter

###  Loading det object instances

In [2]:
det_file_path = '../data/det/det.txt'
gt_gile_path = '../data/gt/gt.txt'
detections_df = load_detections(det_file_path)
detections_df.head()

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1


In [3]:
# Convert conf column to float
detections_df['conf'] = detections_df['conf'].astype(float)

#### Interface (main loop)

In [4]:
# Main loop
image_frames_path = '../data/img1/' 

# Define properties for the output video
fps = 40
frame_size = (1920, 1080) # Adjust this to match the size of your frames
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec
out = cv2.VideoWriter('output_with_tracking.mp4', fourcc, fps, frame_size)
delay = int(1000/fps)

# Initialize tracking variables
tracks_history = {}
frame_number = 1
sigma = 0.2
CONF_THRESH = 28.0
nb_step = 6

init_model()

tracking_file = 'track.txt'
if os.path.isfile(tracking_file):
    open(tracking_file, 'w').close()
else:
    open(tracking_file, 'x')

# Process each image frame
for filename in sorted(os.listdir(image_frames_path)):
    if filename.endswith(".jpg"):
        frame_path = os.path.join(image_frames_path, filename)
        frame = cv2.imread(frame_path)

        # Looking forward
        tracks_history = look_forward(
            detections_df,
            tracks_history,
            frame_number,
            frame,
            image_frames_path,
            nb_step=nb_step,
            sigma=sigma,
            conf_threshold=CONF_THRESH
        )
        
        # Draw tracking
        frame_with_tracking = draw_tracking_result(frame, tracks_history[frame_number])
        # Save the tracking results
        save_tracking_results(tracks_history[frame_number], tracking_file, frame_number)

        # Show the frame
        cv2.imshow('Tracking', frame_with_tracking)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
        out.write(frame_with_tracking)

        frame_number += 1

out.release()
cv2.destroyAllWindows()

/home/coartix/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/coartix/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0.2130305388661763
0.20148162859216426
0.20760898637550748
0.20964199410628476
0.21194667732151506
0.21743621455624457
0.22422032964740465
0.2328337308374645
0.23996124099704846
0.24059280695494592
0.23506238234836344
0.2307329675885135
0.22507184774646152
0.22137544086082478
0.2144193261862843
0.20996388943922692
0.20683382769633377
0.20454408171132515
0.20185713778941722
0.19593393265757733
0.19377231512992743
0.19540945758633424
0.19475579261752332
0.196397034866932
0.1956645936346155
0.1953811448945672
0.19573129272810186
0.19472533187834987
0.19509550387331037
0.19617856760437355
0.1985735696965829
0.20177812042579207
0.205057393371796
0.20649357242941918
0.2083722202756264
0.20892438591538748
0.21072046955023702
0.21235514284534962
0.21373688741522862
0.2151565916051467
0.21705749810133984
0.2190932991758011
0.2210558225628146
0.22272411148949786
0.2244943686534971
0.22589836572582545
0.2275108112541503
0.22907226026287222
0.23057473890003896
0.23156052161202959
0.232393669935629

KeyboardInterrupt: 